In [3]:
import numpy as np
from pyfigtree import figtree
from sklearn import preprocessing

In [4]:
def direct_evaluation(S,T,weights,delta):
    checkPot = []
    for target in T:
        # for each source, add to potential at current target
        diff = target-S
        norm = np.linalg.norm(diff, axis=1)
        up = (norm / delta)**2
        potential = np.exp(-up)*weights
        
        cum_potential = np.sum(potential)
        checkPot.append(cum_potential)
    return np.array(checkPot)

In [5]:
delta = 1


#Direct Evaluation Test with sphere data
X = np.loadtxt('data/sphere_x.txt')
Y = np.loadtxt('data/sphere_y.txt')
Z = np.loadtxt('data/sphere_z.txt')

X0 = np.dstack((X,Y,Z))[0]

X = np.loadtxt('data/alpha0_x.txt')
Y = np.loadtxt('data/alpha0_y.txt')
Z = np.loadtxt('data/alpha0_z.txt')

alpha0 = np.dstack((X,Y,Z))[0]
weights = alpha0

In [6]:
cp_x = figtree(X0,X0,alpha0[:,0], delta,epsilon=1e-16, eval="auto")
cp_y = figtree(X0,X0,alpha0[:,1], delta,epsilon=1e-16, eval="auto")
cp_z = figtree(X0,X0,alpha0[:,2], delta,epsilon=1e-16, eval="auto")
cp = np.dstack([cp_x,cp_y, cp_z])[0]

# Form Evolution Equations
## Direct Functions

In [7]:
def direct_position(X,alpha,delta):
    X_x = direct_evaluation(X,X,alpha[:,0], delta)
    X_y = direct_evaluation(X,X,alpha[:,1], delta)
    X_z = direct_evaluation(X,X,alpha[:,2], delta) 
    return np.dstack([X_x,X_y, X_z])[0]

In [8]:
def direct_momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = direct_evaluation(X,X,first_weights[:,0],delta)
    first_sum_y = direct_evaluation(X,X,first_weights[:,1],delta)
    first_sum_z = direct_evaluation(X,X,first_weights[:,2],delta)
    
    
    second_sum_x = direct_evaluation(X,X,second_weights[:,0],delta)
    second_sum_y = direct_evaluation(X,X,second_weights[:,1],delta)
    second_sum_z = direct_evaluation(X,X,second_weights[:,2],delta)
    
    
    third_sum_x = direct_evaluation(X,X,third_weights[:,0],delta)
    third_sum_y = direct_evaluation(X,X,third_weights[:,1],delta)
    third_sum_z = direct_evaluation(X,X,third_weights[:,2],delta)
    
    fourth_sum = direct_evaluation(X,X,ax,delta)
    fifth_sum = direct_evaluation(X,X,ay,delta)
    sixth_sum = direct_evaluation(X,X,az,delta)
    
    second_term = fourth_sum + fifth_sum + sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
    
    
    mult = -2./X.shape[0]
    result_x = mult*(ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) )
    result_y = mult*(ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) )
    result_z = mult*(ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z) )
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

## Figtree Functions

In [9]:
def position(X,alpha,delta):
    X_x = figtree(X,X,alpha[:,0], delta,epsilon=1e-16, eval="auto")
    X_y = figtree(X,X,alpha[:,1], delta,epsilon=1e-16, eval="auto")
    X_z = figtree(X,X,alpha[:,2], delta,epsilon=1e-16, eval="auto") 
    return np.dstack([X_x,X_y, X_z])[0]

In [10]:
def momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = figtree(X,X,first_weights[:,0],delta, epsilon=1e-16, eval="auto")
    first_sum_y = figtree(X,X,first_weights[:,1],delta, epsilon=1e-16, eval="auto")
    first_sum_z = figtree(X,X,first_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    second_sum_x = figtree(X,X,second_weights[:,0],delta, epsilon=1e-16, eval="auto")
    second_sum_y = figtree(X,X,second_weights[:,1],delta, epsilon=1e-16, eval="auto")
    second_sum_z = figtree(X,X,second_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    third_sum_x = figtree(X,X,third_weights[:,0],delta, epsilon=1e-16, eval="auto")
    third_sum_y = figtree(X,X,third_weights[:,1],delta, epsilon=1e-16, eval="auto")
    third_sum_z = figtree(X,X,third_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    fourth_sum = figtree(X,X,ax,delta, epsilon=1e-16, eval="auto")
    fifth_sum = figtree(X,X,ay,delta, epsilon=1e-16, eval="auto")
    sixth_sum = figtree(X,X,az,delta, epsilon=1e-16, eval="auto")
    
    second_term = fourth_sum + fifth_sum + sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
 
    mult = -2./X.shape[0]
    result_x = mult*(ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) )
    result_y = mult*(ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) )
    result_z = mult*(ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z) )
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

## Testing

In [11]:
delta = 1

In [12]:
%time postition1 = direct_position(X0,alpha0,delta)

CPU times: user 3.75 s, sys: 0 ns, total: 3.75 s
Wall time: 3.68 s


In [16]:
%time postition2 = position(X0,alpha0,delta)

CPU times: user 172 ms, sys: 0 ns, total: 172 ms
Wall time: 164 ms


In [14]:
%time momentum1 = direct_momentum(X0,alpha0,delta)
print momentum1
print np.linalg.norm(np.sum(momentum1, axis = 0))

CPU times: user 13.7 s, sys: 0 ns, total: 13.7 s
Wall time: 13.6 s
[[-0.00435011 -0.00434019 -0.00328521]
 [-0.00098886 -0.00217136 -0.00164537]
 [ 0.00227232  0.00397511  0.00236348]
 ..., 
 [-0.00408169 -0.00418028 -0.00235179]
 [-0.49343779 -0.4699045  -0.33524342]
 [-0.74304366 -0.6981588  -0.50786343]]
1.96334195181


In [15]:
%time momentum2 = momentum(X0,alpha0,delta)
print momentum2
print np.linalg.norm(np.sum(momentum2, axis = 0))

CPU times: user 648 ms, sys: 0 ns, total: 648 ms
Wall time: 639 ms
[[-0.00435011 -0.00434019 -0.00328521]
 [-0.00098886 -0.00217136 -0.00164537]
 [ 0.00227232  0.00397511  0.00236348]
 ..., 
 [-0.00408169 -0.00418028 -0.00235179]
 [-0.49343779 -0.4699045  -0.33524342]
 [-0.74304366 -0.6981588  -0.50786343]]
1.96334195181


In [48]:
error_pos = np.sum(np.abs(postition1-postition2))
error_pos

2.6962254651152762e-10

In [49]:
error_mom = np.sum(np.abs(momentum1-momentum2))
error_mom

1.9347049570045405e-11

In [51]:
def f(vector, delta):
    positions, momentums = np.split(vector,2)
    positions = np.reshape(positions,(3,len(positions)/3)).T
    momentums = np.reshape(momentums,(3,len(momentums)/3)).T
    new_positions = position(positions, momentums,delta)
    new_momentums = momentum(positions, momentums,delta)
    n,m = new_positions.shape
    new_positions = np.reshape(new_positions.T, (n*m))
    new_momentums = np.reshape(new_momentums.T, (n*m))
    return np.hstack([new_positions,new_momentums])

In [59]:
X = np.loadtxt('data/sphere_x.txt')
Y = np.loadtxt('data/sphere_y.txt')
Z = np.loadtxt('data/sphere_z.txt')

X1 = np.hstack((X,Y,Z))

A = np.loadtxt('data/alpha0_x.txt')
B = np.loadtxt('data/alpha0_y.txt')
C = np.loadtxt('data/alpha0_z.txt')

alpha1 = np.hstack((A,B,C))

vector = np.hstack([X1,alpha1])
vector1 = f(vector, delta)

positions, momentums = np.split(vector1,2)
positions = np.reshape(positions,(3,len(positions)/3)).T
momentums = np.reshape(momentums,(3,len(momentums)/3)).T
print np.sum(np.abs(postition2-positions))
print np.sum(np.abs(momentum2-momentums))

0.0
0.0
